# Utility for plotting dat files containing narrowband detections.


#### Misc imports, system variables and functions

In [ ]:
import sys
import os
import subprocess

import matplotlib.pyplot as plt
%matplotlib inline
params = {'legend.fontsize': 'medium',
          'figure.figsize': (10,6),
         'axes.labelsize': 'large',
         'axes.titlesize':'large',
         'xtick.labelsize':'large',
         'ytick.labelsize':'large'}
plt.rcParams.update(params)

import src.plot_fns as pltg             # generic plot fns
import src.plot_h5_psd_sg1 as plt_h5    # blimpy-based plot fns

import math
import numpy as np
from astropy import units as u
import blimpy as bl
import time
import pandas

from pathlib import Path

# sys.path.append(os.getenv('SETIGEN_PATH'))

# import setigen as stg

raw_dir = os.getenv('RAWDIR') + '/'
if not os.path.isdir(raw_dir[0:-1]):
    os.system('mkdir '+raw_dir[0:-1])

raw_backup_base_dir = os.getenv('RAW_BACKUP_BASE_DIR') + '/'

sg_dir = os.getenv('SGDIR') + '/'
if not os.path.isdir(sg_dir[0:-1]):
    os.system('mkdir '+sg_dir[0:-1])

output_dir = os.getenv('OUTDIR') + '/'
if not os.path.isdir(output_dir[0:-1]):
    os.system('mkdir '+output_dir[0:-1])

def db(x):
    """ Convert linear value to dB value """
    return 10*np.log10(np.abs(x.astype(np.float64))+1e-20)



In [ ]:
try:
    parameters_are_undefined
except NameError:
    parameters_are_undefined = True     
    print('Parameters are undefined, using defaults\n')


In [ ]:
# Sometimes it can be necessary to re-run this command for plots to show automatically
%matplotlib inline

#### Parameter setup for seticore, bliss or turboseti runs
Need to be sure proper seticore2 or bliss2 branch is set up and compiled (e.g. "git checkout sc2", "git status", "meson compile"), and indicated here

Note: test_case appears in plots and plot names, and can be set to the branch name or arbitrary string

In [ ]:
if parameters_are_undefined:
    display_figs01 = True
    plot_all_dets = True
    plot_multichirp_dets1 = True
    plot_multichirp_dets2 = True
    verbose = True
    plot_psd_zoom = False
    plot_sg_zoom = False
    plot_sg_wb = False
    plot_psd_wb = True

    plot_sg_psd_zoom = False
    zoom_bw_MHz = .100   # +/- 50 KHz
    max_n_zoom_plots = 20
    zoom_min_max_freq_MHz = []
    zoom_min_max_drift_rate = []

    search_max_drift = 10    # Hz/sec
    search_min_drift = -10   # ignored except for plotting

    fig_dir = 'plots/'

    if (1):
        dat_backup_subdir = ''
        dat_name = 'testout.dat'
        # dat_name = 'gbt-chirp80-0.0020V-6002.20-6003.80-m10.0-10.0-Hzsec-45.81sec-1024K-Nsti-16.chnl_1024K_12_256ch.dat'
        # dat_name = 'gbt-chirp80-0.0020V-6002.20-6003.80-m10.0-10.0-Hzsec-45.81sec-1024K-Nsti-16.chnl_1024K_12_8ch.dat'
        # dat_name = 'gbt-chirp80-0.0020V-6002.20-6003.80-m10.0-10.0-Hzsec-45.81sec-1024K-Nsti-16.chnl_1024K_8_8ch.dat'
        # dat_name = 'gbt-chirp80-0.0020V-6002.20-6003.80-m10.0-10.0-Hzsec-45.81sec-1024K-Nsti-16.chnl_1024K_8_256ch.dat'

full_dat_name = output_dir + dat_name

dat_file_stem = dat_name.split('.dat')[0]


In [ ]:
from src.find_event import read_dat
try:
    det_list = read_dat(full_dat_name)
except:
    print('Error in reading dat file')
    assert(False)    

In [ ]:
if (not det_list.empty):
    det_list = det_list.sort_values(by='Freq')

In [ ]:
# det_list

In [ ]:
obs_length = float(det_list.obs_length[0])
delta_f = float(det_list.DELTAF[0])
delta_t = float(det_list.DELTAT[0])
det_snr_db = db(det_list.SNR)
det_f_start_MHz = det_list.FreqStart
det_drift_rate = det_list.DriftRate
search_max_drift = float(det_list.max_drift_rate[0])
full_h5_file_name = det_list.FileID[0]
Source = det_list.Source[0]
MJD = det_list.MJD[0]
RA = det_list.RA[0]
DEC = det_list.DEC[0]

search_min_drift = -search_max_drift

n_sti = round(delta_t*delta_f)
n_lti = round(obs_length/delta_t)
h5_file_name = full_h5_file_name.split('/')[-1]

n_det = len(det_snr_db)

In [ ]:
whos

### Plot detection results - All coarse channels, or just channel with test chirps

In [ ]:
f1_MHz = min(det_f_start_MHz)-1.
f2_MHz = max(det_f_start_MHz)+1.
drift1 = search_min_drift-1
drift2 = search_max_drift+1

fig = plt.figure(figsize=(10, 6))

plt.subplot(2,1,1)
pltg.plot_generic(x_data=[det_f_start_MHz],y_data=[det_drift_rate],
    fig = fig,
    xy_markers = 'g*',
    x_limits=[f1_MHz,f2_MHz],
    y_limits=[drift1,drift2],
    x_label = '',
    y_label = 'Drift Rate Hz/sec',
    fig_title= 'Drifting Tone Detections',
    fig_text_list=[[.14,.85,dat_name],[.14,.82,f'Hit Count:{n_det}']])

plt.subplot(2,1,2)

pltg.plot_generic(x_data=[det_f_start_MHz],y_data=[det_snr_db],
    fig = fig,
    xy_markers = 'g*',
    x_limits=[f1_MHz,f2_MHz],
    y_limits=[5.,40.],
    x_label = 'Frequency MHz',
    y_label = 'SNR dB')

plt.savefig(fig_dir+'06-'+dat_file_stem+'.png',bbox_inches='tight')
plt.show()
